In [1]:
# neural network classification with pytorch
# Make classification data
import sklearn
from sklearn.datasets import make_circles

In [2]:
# make 1000 samples
n_samples = 1000

# create circles
X, y = make_circles(n_samples, noise=0.03, random_state=42)


In [4]:
len(y)

1000